In [8]:
import pandas as pd
from pandas.io import gbq
import datetime
from datetime import date

# Import the mapping tables

subsidiaries = pd.read_csv('/home/markaw/Heroes/ns_cashsales/subsidiaries.csv')
customers = pd.read_csv('/home/markaw/Heroes/ns_cashsales/Customer.csv', encoding='iso-8859-1')
heroesID =  pd.read_csv(r'/home/markaw/Heroes/ns_cashsales/heroesID.csv', encoding='iso-8859-1')

# provide variables for BQ extract

start_dt = input('please provide a start date yyyy/mm/dd')
end_dt = input('please provide an end date yyyy/mm/dd')

while True:
    brand = input('please provide Brand')
    if brand not in subsidiaries['Brand'].tolist():
        print('Please check the spelling for that brand')
        continue
    else:
        break

while True:
    customer = input('please provide Customer')
    if customer not in customers['Customer reference'].tolist():
        print('That customer seems incorrect, check spelling')
        continue
    else:
        break

# tranId = input('please provide a tranId, eg CS100')
        
# Run BQ extract 

sql_query = f"""
SELECT * 
FROM analytics-298917.analytics_models.f_amz_order_items

WHERE purchase_dt BETWEEN '{start_dt}' AND '{end_dt}'
"""
bq_raw = gbq.read_gbq(sql_query, project_id = "analytics-298917")
bq = bq_raw[(bq_raw['order_status'] != 'Cancelled') & 
            (bq_raw['sales_channel'] != 'Non-Amazon') &
            (bq_raw['quantity'] != 0)].reset_index()


# Create a draft NetSuite CashSale import template

columns = ['tranId','customerTEMP','tranDate','Mon','Year','currencyRef','exchangeRate',
'isTaxable','itemLine_itemRefTEMP','itemLine_quantity','itemLine_unitsRef','itemLine_salesPrice','itemLine_priceLevelRef',
'Subsiduary','Expected Deposit date']

template = pd.DataFrame(columns=columns)

# Bring BQ data into the template

template['customerTEMP'] = bq['sales_channel']
template['tranDate'] = bq['purchase_dt'].apply(lambda x: x.strftime("%d/%m/%Y"))
template['Mon'] = bq['purchase_dt'].apply(lambda x: x.strftime("%b")[0:3])
template['Year'] = bq['purchase_dt'].apply(lambda x: x.year)
template['currencyRef'] = bq['currency']
template['exchangeRate'] = 1
template['isTaxable'] = 'TRUE'
template['itemLine_itemRefTEMP'] = bq['asin']
template['itemLine_quantity'] = bq['quantity']
template['itemLine_unitsRef'] = 'Units'
template['itemLine_salesPrice']= bq['item_price_local']/bq['quantity']
template['itemLine_priceLevelRef'] = 'Custom'
template['Subsiduary'] = bq['brand_name']
template['Expected Deposit date'] = (bq['purchase_dt'].max()+datetime.timedelta(days=15)).strftime("%d/%m/%Y")
template['tranId'] = tranId


# Merge template with information from the mapping tables

merged = pd.merge(template,subsidiaries, left_on='Subsiduary', right_on='Brand', how='inner')
merged1 = pd.merge(merged,customers, left_on='customerTEMP', right_on='Sales channel', how='inner')
merged2 = pd.merge(merged1,heroesID, left_on='itemLine_itemRefTEMP', right_on='ASIN', how='inner')
merged2['locationRef'] = 'Amazon FBA - ' + merged2['Acronym'] +' - '+ merged2['Country Code']

# Filter by desired customer ID

customer_filter = merged2[(merged2['Customer reference'] == customer) & (merged2['Brand'] == brand)].copy()

# Modify country ticker for US/UK or else the will both populate the template as UNI

if customer == '14 Amazon - UK':
    customer_filter['Country'] = 'UK'
elif customer == '16 Amazon - US':
    customer_filter['Country'] = 'US'

# Select the columns we need and re-make the final import template

final_dict = {
    'ExternalID': f"""{customer_filter.iloc[0][13][0:3].upper()}{date.today().strftime("%d%m%Y")}{customer_filter.iloc[0][19][0:3].upper()}""",
    'tranId':' ',
    'customerRef':customer_filter['Customer reference'],
    'tranDate':customer_filter['tranDate'],
    'postingPeriodRef':customer_filter['Mon'] +" "+customer_filter['Year'].astype(str),
    'currencyRef':customer_filter['currencyRef'],
    'exchangeRate':customer_filter['exchangeRate'],
    'locationRef':customer_filter['locationRef'],
    'isTaxable':customer_filter['isTaxable'],
    'itemLine_itemRef':customer_filter['NEW (FINAL) HEROES ID'],
    'itemLine_quantity':customer_filter['itemLine_quantity'],
    'itemLine_unitsRef':customer_filter['itemLine_unitsRef'],
    'itemLine_salesPrice':customer_filter['itemLine_salesPrice'],
    'itemLine_priceLevelRef':customer_filter['itemLine_priceLevelRef'],
    'Subsiduary':customer_filter['Code'],
    'Expected Deposit date':customer_filter['Expected Deposit date'],
}

# Create the final dataframe template

final = pd.DataFrame(final_dict).reset_index()

# Export 'final' to csv, splitting it if >5k (NetSuite import limit)

if len(final) < 5000:
    name = f'/home/markaw/Heroes/ns_cashsales/CS Templates/{customer_filter.iloc[0][13][0:3].upper()}{date.today().strftime("%d%m%Y")}{customer_filter.iloc[0][19][0:3].upper()}.csv'
    final.to_csv(name, index=False)
    
elif 5000 <= len(final) < 10000:
    name = f'/home/markaw/Heroes/ns_cashsales/CS Templates/{customer_filter.iloc[0][13][0:3].upper()}{date.today().strftime("%d%m%Y")}{customer_filter.iloc[0][19][0:3].upper()}1.csv'
    final[0:5000].to_csv(name, index=False)
    
    name2 = f'/home/markaw/Heroes/ns_cashsales/CS Templates/{customer_filter.iloc[0][13][0:3].upper()}{date.today().strftime("%d%m%Y")}{customer_filter.iloc[0][19][0:3].upper()}2.csv' 
    final2 = final[(final.index >= 5001) & ((final.index < 10000))].copy()
    final2['ExternalID'] = final['ExternalID']+'b'
#     final2['tranId'] = "CS"+str(int(final['tranId'][0][-3:])+1)
    final2.to_csv(name2, index=False)
    
elif len(final) >= 10000:
    name = f'/home/markaw/Heroes/ns_cashsales/CS Templates/{customer_filter.iloc[0][13][0:3].upper()}{date.today().strftime("%d%m%Y")}{customer_filter.iloc[0][19][0:3].upper()}1.csv'
    final[0:5000].to_csv(name, index=False)
    
    name2 = f'/home/markaw/Heroes/ns_cashsales/CS Templates/{customer_filter.iloc[0][13][0:3].upper()}{date.today().strftime("%d%m%Y")}{customer_filter.iloc[0][19][0:3].upper()}2.csv' 
    final2 = final[(final.index >= 5001) & ((final.index < 10000))].copy()
    final2['ExternalID'] = final['ExternalID']+'b'  
#     final2['tranId'] = "CS"+str(int(final['tranId'][0][-3:])+1)
    final2.to_csv(name2, index=False)
    
    name3 = f'/home/markaw/Heroes/ns_cashsales/CS Templates/{customer_filter.iloc[0][13][0:3].upper()}{date.today().strftime("%d%m%Y")}{customer_filter.iloc[0][19][0:3].upper()}3.csv' 
    final3 = final[(final.index > 10000)].copy()
    final3['ExternalID'] = final['ExternalID']+'c' 
#     final3['tranId'] = "CS"+str(int(final['tranId'][0][-3:])+2) 
    final3.to_csv(name3, index=False)


print(f'{len(final)} rows exported! Check your folder!')


please provide a start date yyyy/mm/dd 2021-05-04
please provide an end date yyyy/mm/dd 2021-05-18
please provide Brand Viridescent
please provide Customer 14 Amazon - UK
please provide a tranId, eg CS100 1


995 rows exported! Check your folder!


In [7]:
final['itemLine_itemRef'].unique()

array(['DAV-TLP-WW-ZZ-68', 'DAV-SPS-WW-ZZ-SE', 'DAV-RLP-WW-ZZ-60',
       'DAV-RHS-WW-ZZ-SE', 'DAV-SHE-WW-ZZ-NA', 'DAV-RHS-WW-ZZ-NA',
       'DAV-RCS-WW-ZZ-NA', 'DAV-SRS-WW-ZZ-NA', 'DAV-HWS-WW-ZZ-NA',
       'DAV-GRS-WW-ZZ-SE', 'DAV-PHS-WW-ZZ-NA', 'DAV-TLP-WW-ZZ-63',
       'DAV-PSW-WW-GN-NA', 'DAV-TSP-WW-ZZ-NA', 'DAV-PSW-WW-RD-NA',
       'DAV-CBS-WW-ZZ-NA', 'DAV-PSW-WW-ZZ-25', 'DAV-RST-WW-ZZ-SE'],
      dtype=object)